# This notebook presents the computation time required to analyze a single HUC12
##### Author: Omid Emamjomehzadeh (https://www.omidemam.com/)
##### Supervisor: Dr. Omar Wani (https://engineering.nyu.edu/faculty/omar-wani)
##### Hydrologic Systems Group @NYU (https://www.omarwani.com/)

In [13]:
#import libraries
import cProfile
import whitebox 
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import colormaps
from matplotlib_scalebar.scalebar import ScaleBar
from contextlib import contextmanager
from shapely.ops import split
from shapely.geometry import LineString, Polygon
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from watermark import watermark
import random
import pyproj
from pyproj import CRS
from datetime import datetime
import requests
import os
import shutil
import sys
import subprocess
import datetime

In [3]:
# Define watershed delineation tool
wbt = whitebox.WhiteboxTools()

In [4]:
# This function stops the White box tool from outputting text in the notebook
@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

In [5]:
#Define layers directories
output_huc12_dir = r'D:\culvert repo\Results\Clipped_layers\clipped_huc12'
output_stream_dir = r'D:\culvert repo\Results\Clipped_layers\clipped_streams'
output_road_dir = r'D:\culvert repo\Results\Clipped_layers\clipped_street'
output_railroad_dir = r'D:\culvert repo\Results\Clipped_layers\clipped_railroad'
output_culvert_dir = r'D:\culvert repo\Results\Clipped_layers\clipped_culverts'
output_DEM_dir=r'D:\culvert repo\Results\Clipped_layers\HUC12_one meter_DEM'

In [6]:
# Read HUC12 list
file_path = r"D:\culvert repo\Results\Lists\huc12_with_all_layers.txt"
with open(file_path, 'r') as file:
    huc12_with_all_layers = [line.strip() for line in file]
total_hucs=len(huc12_with_all_layers)    

In [25]:
selected_hucs=huc12_with_all_layers[:1]
selected_hucs

['050100020104']

In [19]:
# Define a function to profile the time
def main(selected_hucs):
    results = []
    huc_analysed_correc=[]
    for i, huc in enumerate(huc12_with_all_layers[:1]):
        # Check if the file exists; if so, skip to the next iteration
        #finalfile=fr'D:\culvert repo\Results\waterhsed_delineation_dir\{huc}\{huc}_watersheds_info.shp'
        #if os.path.exists(finalfile):
        #    print(f"File {finalfile} already exists. Skipping...")
        #    continue
        # Check if the file exists; if so, skip to the next iteration
        #finalfile=fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}\{huc}_watersheds_info.shp'
        #if os.path.exists(finalfile):
        #    print(f"File {finalfile} already exists. Skipping...")
        #    continue
        # Check if the DEM file exists; if not, skip to the next iteration
        clipped_tiff = os.path.join(rf'D:\culvert repo\Results\Clipped_layers\HUC12_one meter_DEM\{huc}', f'{huc}_DEM.tiff')
        if not  os.path.exists(clipped_tiff):
            print(f"DEM File {clipped_tiff} doenst exist. Skipping...")
            continue
        try:
            print(huc)
            # Print the HUC number and the percent of remaining HUCs in this chunk
            remaining_files = total_hucs - (i + 1)
            percent_remaining = (remaining_files / total_hucs) * 100
            print(f"HUC number: {huc}, Percent of remaining files: {percent_remaining:.2f}%")
            # path to the specific HUC layer
            output_huc12_path = os.path.join(output_huc12_dir, f'{huc}_huc12.shp')
            output_stream_path = os.path.join(output_stream_dir, f'{huc}_streams.shp')
            output_culvert_path = os.path.join(output_culvert_dir, f'{huc}_culverts.shp')
            output_road_path = os.path.join(output_road_dir, f'{huc}_streets.shp')
            output_railroad_path=os.path.join(output_road_dir, f'{huc}_railroads.shp')
            clipped_tiff = os.path.join(rf'D:\culvert repo\Results\Clipped_layers\HUC12_one meter_DEM\{huc}', f'{huc}_DEM.tiff')
            # Load data using Geopandas
            huc12 = gpd.read_file(output_huc12_path)
            streams = gpd.read_file(output_stream_path)
            culverts = gpd.read_file(output_culvert_path)
            roads = gpd.read_file(output_road_path)
            if os.path.exists(output_railroad_path):
                railroads=gpd.read_file(output_road_path)
            ################################################################################################################################################ 
            # make a directory for watershed delineation analysis
            # Define and create the directory
            water_deal_dir_path= fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}'
            # Create the directory if it doesn't exist
            if not os.path.exists(water_deal_dir_path):
                os.makedirs(water_deal_dir_path)
            # Set the working directory
            wbt.set_working_dir(water_deal_dir_path)
            ################################################################################################################################################
            # Lowers the elevation of roads to ground elevation when there is an intersection of stream and roads or railroads
            #(this helps to generate realistic stream flow that accounts for the existence of culverts)
            # fill DEM for slope calculation
            # Generat a Dem file for this calcualtion which is filled but is not burned
            mo_dem_5 = fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}\{huc}_DEM_mo_5.tif'
            with suppress_stdout():
                wbt.fill_depressions(
                    clipped_tiff, 
                    mo_dem_5, 
                    fix_flats=True, 
                    flat_increment=None, 
                    max_depth=None, 
                )
            ################################################################################################################################################
            roadsburned_dem = fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}\{huc}_DEM_roadsburned.tif'
            with suppress_stdout():
                wbt.burn_streams_at_roads(
                    clipped_tiff, 
                    output_stream_path, 
                    output_road_path, 
                    roadsburned_dem, 
                    width=60, 
                )
            if os.path.exists(output_railroad_path):
                with suppress_stdout():
                    wbt.burn_streams_at_roads(
                        roadsburned_dem, 
                        output_stream_path, 
                        output_road_path, 
                        roadsburned_dem, 
                        width=60, 
                    )
            ################################################################################################################################################
            mo_dem_4 = fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}\{huc}_DEM_mo_4.tif'
            with suppress_stdout():
                wbt.fill_depressions(
                    roadsburned_dem, 
                    mo_dem_4, 
                    fix_flats=True, 
                    flat_increment=None, 
                    max_depth=None, 
                )
            ################################################################################################################################################
            # Generate Flow direction raster file
            flowdirection=  fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}\{huc}_flowdir.tif'
            with suppress_stdout():
                wbt.d8_pointer(
                    mo_dem_4, 
                    flowdirection, 
                    esri_pntr=False, 
                )
            ################################################################################################################################################
            # Generate Flow accumulation raster file
            flowacc=  fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}\{huc}flowacc.tif'
            with suppress_stdout():
                wbt.d8_flow_accumulation(
                    mo_dem_4, 
                    flowacc, 
                    out_type="cells", 
                    log=False, 
                    clip=False, 
                    pntr=False, 
                    esri_pntr=False, 
                )
            ################################################################################################################################################
            # Define the culvert_data, which will contain all information from this analysis
            culvert_data = []
            # Define distances
            distances = [ 10, 20, 40, 80]
            for index, row in culverts.iterrows():
                # Construct the filename for each row
                filename = f"{row['BIN']}.shp"
                # Save each row as a shapefile in the output directory
                culvert_output_path = os.path.join(water_deal_dir_path, filename)
                culvert = gpd.GeoDataFrame([row], geometry='geometry',crs=culverts.crs)
                culvert.to_file(culvert_output_path)
                for distance in distances:
                    sanppoints_path = fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}\{row["BIN"]}snap_pour_point_{distance}m.shp'
                    with suppress_stdout():
                        wbt.snap_pour_points(
                            culvert_output_path, 
                            flowacc, 
                            sanppoints_path, 
                            snap_dist=distance, 
                        )
                    sanpped_points=gpd.read_file(sanppoints_path)
                    sanpped_points = sanpped_points.set_crs(culverts.crs, allow_override=True)
                    ########################################################################################################################################
                    # Watershed delineation based on snapped culvert points
                    watershed=fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}\{culvert["BIN"].iloc[0]}watershed_{distance}m.tif'
                    with suppress_stdout():
                        wbt.watershed(
                            flowdirection, 
                            sanppoints_path, 
                            watershed, 
                            esri_pntr=False, 
                        )
                    # Path to the output vector file
                    watershed_polygone = fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}\{culvert["BIN"].iloc[0]}watershed_poly_{distance}m.shp'
                    with suppress_stdout():
                        # Run the RasterToVectorPolygons tool
                        wbt.raster_to_vector_polygons(watershed, watershed_polygone)
                    ########################################################################################################################################
                    # longest flow path calculation
                    lfp=fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}\{culvert["BIN"].iloc[0]}_lfp_{distance}m.shp'
                    with suppress_stdout():
                        wbt.longest_flowpath(
                            mo_dem_4, 
                            watershed, 
                            lfp, 
                        )
                    ########################################################################################################################################
                    #load the watershed and longest flow path data frames
                    watershed_gdf=gpd.read_file(watershed_polygone)
                    lfp_gdf=gpd.read_file(lfp)
                    # Find the maximum flow path length and average slope
                    max_length_rows = lfp_gdf.loc[lfp_gdf.groupby('BASIN')['LENGTH'].idxmax()]
                    ########################################################################################################################################
                    # calculate the average slope of each basin
                    # Clip and save the reprojected DEM
                    clipped_tiff_watershed = os.path.join(rf'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}',
                                                          f'{culvert["BIN"].iloc[0]}_DEM_watershed_.tiff')
                    # load watershed polygon
                    watershed_pol=gpd.read_file(watershed_polygone)
                    # Convert the HUC12 boundary to GeoJSON-like format
                    watershed_pol_geom = [feature["geometry"] for feature in watershed_pol.__geo_interface__["features"]]
                    # Open the reprojected TIFF file and clip it using the HUC12 boundary
                    with rasterio.open(clipped_tiff) as src:
                        out_image, out_transform = mask(src, watershed_pol_geom, crop=True)
                        # Set NoData value
                        out_image[out_image == src.nodata] = -99999
                        out_meta = src.meta.copy()
                        out_meta.update({
                            "driver": "GTiff",
                            "height": out_image.shape[1],
                            "width": out_image.shape[2],
                            "transform": out_transform,
                            "crs": src.crs,
                            "nodata": -99999
                        })
                        # Save the clipped DEM to a new file
                        with rasterio.open(clipped_tiff_watershed, 'w', **out_meta) as dst:
                            dst.write(out_image)
                    # calculate the average slope
                    # Define input and output file paths
                    output_slope = fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}\{culvert["BIN"].iloc[0]}_slope.tif'
                    # Run the Slope tool with units in percent
                    with suppress_stdout():
                        wbt.slope(dem=clipped_tiff_watershed, output=output_slope, zfactor="1.0", units="percent")
                    # Read the slope raster
                    with rasterio.open(output_slope) as slope_dataset:
                        slope_data = slope_dataset.read(1)
                    # Mask out NoData values
                    slope_data = np.ma.masked_equal(slope_data, slope_dataset.nodata)
                    # Calculate the average slope
                    average_slope = np.mean(slope_data)
                    ########################################################################################################################################
                    # Calculat the slope of the stream near the culvert outlet as a proxy of the culvert slope
                    def sample_dem_at_points(dem_src, points_gdf):
                        coords = [(x,y) for x, y in zip(points_gdf.geometry.x, points_gdf.geometry.y)]
                        sampled_values = list(dem_src.sample(coords))
                        return [val[0] for val in sampled_values]
                    # Extract the specific pour point geometry
                    pour_point = sanpped_points.geometry.iloc[0]
                    # define the longest flow path 
                    longest_lfp = max_length_rows.geometry.iloc[0]
                    # Distances to evaluate the slope of different lengeths of the stream
                    distances_to_evaluate = [20+distance]
                    output_dir = fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}'
                    # Extract start and end points from 'lfp_gdf'
                    end_points = lfp_gdf.geometry.apply(lambda geom: geom.interpolate(1, normalized=True))
                    end_gdf = gpd.GeoDataFrame(geometry=end_points, crs=lfp_gdf.crs)
                    # Save end point shapefile
                    end_shp_path = os.path.join(output_dir, f'{culvert["BIN"].iloc[0]}_{distance}_end_lfp.shp')
                    end_gdf.to_file(end_shp_path)
                    # Open DEM file
                    with rasterio.open(mo_dem_5) as dem_src:
                        dem_data = dem_src.read(1)  # Read the first band
                        dem_nodata = dem_src.nodata  # Get the NoData value from the raster metadata
                        dem_bounds = dem_src.bounds
                        # Replace NoData values with NaN for processing
                        dem_data = np.where(dem_data == dem_nodata, np.nan, dem_data)
                        dem_transform = dem_src.transform
                        # Initialize an empty list to store all differences
                        all_slope_perc = []
                        # Loop through the distances to evaluate
                        for distance_lfp in distances_to_evaluate: 
                            # Interpolate points 'distance' meters from the end of each line
                            distance_m_from_end = lfp_gdf.geometry.apply(lambda geom: geom.interpolate(geom.length - distance_lfp))
                            # Convert the extracted points to a GeoDataFrame
                            distance_m_gdf = gpd.GeoDataFrame(geometry=distance_m_from_end, crs=lfp_gdf.crs)
                            # Define file paths for shapefiles
                            distance_m_shp_path = os.path.join(output_dir, f'{culvert["BIN"].iloc[0]}_{distance}_from_end_lfp_{distance_lfp}.shp')
                            # Save the points as shapefiles
                            distance_m_gdf.to_file(distance_m_shp_path)
                            # Sample DEM at points and compute elevation differences
                            slope_perc = ((np.array(sample_dem_at_points(dem_src, distance_m_gdf)) - 
                                          np.array(sample_dem_at_points(dem_src, end_gdf))) / distance_lfp)*100
                            # Append the results to the list
                            all_slope_perc.append(slope_perc)
                    
                    #######################################################################################################################################
                    # save some variables
                    if distance==10:
                        sanpped_points_10=sanpped_points
                        watershed_gdf_10=watershed_gdf
                        max_length_rows_10=max_length_rows
                        average_slope_10=average_slope
                        average_slope_cul_10=all_slope_perc[0][0]
                    if distance==20:
                        sanpped_points_20=sanpped_points
                        watershed_gdf_20=watershed_gdf
                        max_length_rows_20=max_length_rows
                        average_slope_20=average_slope
                        average_slope_cul_20=all_slope_perc[0][0]
                    if distance==40:
                        sanpped_points_40=sanpped_points
                        watershed_gdf_40=watershed_gdf
                        max_length_rows_40=max_length_rows
                        average_slope_40=average_slope
                        average_slope_cul_40=all_slope_perc[0][0]
                    if distance==80:
                        sanpped_points_80=sanpped_points
                        watershed_gdf_80=watershed_gdf
                        max_length_rows_80=max_length_rows
                        average_slope_80=average_slope
                        average_slope_cul_80=all_slope_perc[0][0]
                    #######################################################################################################################################
                    #remove the single culvert shapefile and watershed and slope raster of each culvert
                    os.remove(clipped_tiff_watershed)
                    os.remove(watershed)
                    os.remove(output_slope)
                ###########################################################################################################################################
                # Store all the data in the HUC_watersheds_info data frame
                culvert_data.append({
                    'BIN': culvert["BIN"].iloc[0],
                    "Ini_lon": culvert.to_crs(epsg=4326).geometry.x.values[0],
                    "Ini_lat": culvert.to_crs(epsg=4326).geometry.y.values[0],
                    'lonsnap_10': sanpped_points_10.to_crs(epsg=4326).geometry.x.values[0],
                    'latsnap_10': sanpped_points_10.to_crs(epsg=4326).geometry.y.values[0],
                    'Area_10': watershed_gdf_10.geometry.area.values[0],
                    'Peri_10': watershed_gdf_10.geometry.length.values[0],
                    'LFP_len_10': max_length_rows_10['LENGTH'].values[0],
                    'LFP_S_10': max_length_rows_10['AVG_SLOPE'].values[0],
                    'Bas_S_10':average_slope_10,
                    'cul_S10':average_slope_cul_10,
                    'geo_10': watershed_gdf_10.geometry.values[0],
                    'lonsnap_20': sanpped_points_20.to_crs(epsg=4326).geometry.x.values[0],
                    'latsnap_20': sanpped_points_20.to_crs(epsg=4326).geometry.y.values[0],
                    'Area_20': watershed_gdf_20.geometry.area.values[0],
                    'Peri_20': watershed_gdf_20.geometry.length.values[0],
                    'LFP_len_20': max_length_rows_20['LENGTH'].values[0],
                    'LFP_S_20': max_length_rows_20['AVG_SLOPE'].values[0],
                    'Bas_S_20':average_slope_20,
                    'cul_S20':average_slope_cul_20,
                    'geo_20': watershed_gdf_20.geometry.values[0],
                    'lonsnap_40': sanpped_points_40.to_crs(epsg=4326).geometry.x.values[0],
                    'latsnap_40': sanpped_points_40.to_crs(epsg=4326).geometry.y.values[0],
                    'Area_40': watershed_gdf_40.geometry.area.values[0],
                    'Peri_40': watershed_gdf_40.geometry.length.values[0],
                    'LFP_len_40': max_length_rows_40['LENGTH'].values[0],
                    'LFP_S_40': max_length_rows_40['AVG_SLOPE'].values[0],
                    'Bas_S_40':average_slope_40,
                    'cul_S40':average_slope_cul_40,
                    'geo_40': watershed_gdf_40.geometry.values[0],
                    'lonsnap_80': sanpped_points_80.to_crs(epsg=4326).geometry.x.values[0],
                    'latsnap_80': sanpped_points_80.to_crs(epsg=4326).geometry.y.values[0],
                    'Area_80': watershed_gdf_80.geometry.area.values[0],
                    'Peri_80': watershed_gdf_80.geometry.length.values[0],
                    'LFP_len_80': max_length_rows_80['LENGTH'].values[0],
                    'LFP_S_80': max_length_rows_80['AVG_SLOPE'].values[0],
                    'Bas_S_80':average_slope_80,
                    'cul_S80':average_slope_cul_80,
                    'geo_80': watershed_gdf_80.geometry.values[0],
                    'geo_cul':culvert.geometry.values[0]
                    })
            ###############################################################################################################################################
            #Save all information about watersheds of the HUC in an Excel file
            # Convert the list to a DataFrame
            culvert_df = pd.DataFrame(culvert_data)
    
            # Define the path where you want to save the Excel file
            excel_file_path = fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}\{huc}_watersheds_info.xlsx'
    
            # Save the DataFrame to an Excel file
            culvert_df.to_excel(excel_file_path, index=False)
    
            print(f"DataFrame saved to {excel_file_path}")
            ###############################################################################################################################################
            #Save all information about watersheds of the HUC in a shapefile
            # Create a data frame from the collected data
            culvert_df = pd.DataFrame(culvert_data)
            #  culvert_df contains the data to be saved as a shapefile
            culvert_gdf = gpd.GeoDataFrame(culvert_df, geometry='geo_40')
            
            # Check if CRS is set, and if not, assign it
            if culvert_gdf.crs is None:
                culvert_gdf.crs = CRS.from_epsg(5070)  # Replace with the appropriate EPSG code
            
            # Drop other redundant columns if needed
            culvert_gdf = culvert_gdf.drop(columns=['geo_10','geo_20', 'geo_80', 'geo_cul'], errors='ignore')
            
            # Path to save the shapefile
            final_HUC_path = fr'C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}\{huc}_watersheds_info.shp'
            
            # Save the GeoDataFrame as a shapefile
            culvert_gdf.to_file(final_HUC_path)
            print(f"Shapefile saved to {final_HUC_path}")
            ###############################################################################################################################################
            # List of file paths to delete
            file_paths = [mo_dem_5,mo_dem_4,flowacc,flowdirection,roadsburned_dem]
        
            try:
                # Iterate over each file path and delete it
                for file_path in file_paths:
                    if os.path.exists(file_path):
                        os.remove(file_path)
                        #print(f"File '{file_path}' deleted successfully.")
                    else:
                        print(f"File '{file_path}' not found.")
            except Exception as e:
                print(f"Error deleting files: {e}")
            ###############################################################################################################################################
            # save all hucs analyzed correctly in a text file
            file_path=r"D:\culvert repo\Results\Lists\huc12s_delineated.txt"
            huc_analysed_correc.append(huc)
            # Open the text file in write mode and save the list
            with open(file_path, 'w') as file:
                for hu in huc_analysed_correc:
                    file.write(f'{hu}\n')
            print("All correctly analysed HUCs written to the text file successfully.")
            ############################################################################################################################################### 
        except Exception as e:
            print(f"Failed to analyse HUC {huc}. Error: {e}")
            shutil.rmtree(rf"C:\hydrological analysis\data\waterhsed_delineation_dir\{huc}")

In [20]:
cProfile.run('main(selected_hucs)')

050100020104
HUC number: 050100020104, Percent of remaining files: 99.92%


C:\Users\Public\anaconda1\envs\culverts\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: C:\hydrological analysis\data\waterhsed_delineation_dir\050100020104\C520186watershed_poly_10m.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(
C:\Users\Public\anaconda1\envs\culverts\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: C:\hydrological analysis\data\waterhsed_delineation_dir\050100020104\C520186watershed_poly_10m.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(
C:\Users\Public\anaconda1\envs\culverts\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: C:\hydrological analysis\data\waterhsed_delineation_dir\050100020104\C520186watershed_poly_20m.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that sh

DataFrame saved to C:\hydrological analysis\data\waterhsed_delineation_dir\050100020104\050100020104_watersheds_info.xlsx
Shapefile saved to C:\hydrological analysis\data\waterhsed_delineation_dir\050100020104\050100020104_watersheds_info.shp
All correctly analysed HUCs written to the text file successfully.
         2619564 function calls (2563216 primitive calls) in 1214.659 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.700    0.700 236049302.py:2(main)
       56    0.001    0.000    0.045    0.001 236049302.py:209(sample_dem_at_points)
       28    0.000    0.000    0.003    0.000 236049302.py:221(<lambda>)
       28    0.000    0.000    0.003    0.000 236049302.py:239(<lambda>)
      290    0.001    0.000    0.010    0.000 4278794996.py:2(suppress_stdout)
      224    0.000    0.000    0.000    0.000 <attrs generated init rasterio._path._ParsedPath>:1(__init__)
       28    0.000    0.

In [22]:
# Date and time
now = datetime.datetime.now()
print(f"Date and time: {now}")

Date and time: 2025-05-12 21:13:01.063679


In [23]:
%load_ext watermark
# Print the Python version and some dependencies
%watermark -v -m -p whitebox,numpy,pandas,geopandas,matplotlib,mpl_toolkits,shapely,os,subprocess,rasterio,watermark,random,py3dep,pyproj,shutil,sys,cProfile

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.20.0

whitebox    : 2.3.5
numpy       : 2.0.2
pandas      : 2.2.2
geopandas   : 1.0.1
matplotlib  : 3.8.4
mpl_toolkits: unknown
shapely     : 2.0.7
os          : unknown
subprocess  : unknown
rasterio    : 1.4.3
watermark   : 2.4.3
random      : unknown
py3dep      : not installed
pyproj      : 3.7.1
shutil      : unknown
sys         : 3.12.4 | packaged by conda-forge | (main, Jun 17 2024, 10:04:44) [MSC v.1940 64 bit (AMD64)]
cProfile    : unknown

Compiler    : MSC v.1940 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 183 Stepping 1, GenuineIntel
CPU cores   : 24
Architecture: 64bit

